In [1]:
#%matplotlib ipympl
import numpy as np
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import os
import openpyxl 
from scipy.integrate import simps
import adi
import pandas as pd
from tqdm import tqdm
from scipy.stats import mannwhitneyu
from scipy.ndimage import gaussian_filter1d

In [2]:
font_path = 'C:\\Windows\\Fonts\\simsun.ttc'  # 宋體
font_prop = FontProperties(fname=font_path)

npy = 'save'
graph = ['break', 'break', 'break']   # 'show' or 'save'
data = 'Normal'

if data == 'Patient':
    data_file = 'F:\\All data\\Patient_Signal'
    graph_file = 'F:\\All data\\Patient_Signal\\graph'
elif data == 'Normal':
    data_file = 'F:\\All data\\Normal_Signal'
    graph_file = 'F:\\All data\\Normal_Signal\\graph'

bad_graph = 'F:\\All data\\Bad_Graph'

In [3]:
def butter(DataL, DataR, cut_low, cut_high, sample_rate, order):
    nyqs = sample_rate * 0.5
    H_cut = cut_high / nyqs
    L_cut = cut_low / nyqs
    sos = signal.butter(order, [L_cut, H_cut], analog=False, btype='bandpass', output='sos')
    Filter_Left = signal.sosfiltfilt(sos, DataL)
    Filter_Right = signal.sosfiltfilt(sos, DataR)
    return Filter_Left, Filter_Right

In [4]:
def bassel(DataL, DataR, cut_low, cut_high, sample_rate, order):
    #貝塞爾
    nyqs = sample_rate * 0.5
    H_cut = cut_high / nyqs
    L_cut = cut_low / nyqs
    sos=signal.bessel(order, [L_cut, H_cut] ,  btype='bandpass',  analog=False,  output='sos')
    Filter_Left = signal.sosfiltfilt(sos,  DataL) 
    Filter_Right = signal.sosfiltfilt(sos,  DataR) 

    return Filter_Left, Filter_Right

In [5]:
def find_peak(Filter_Data):
    valley_x, valley_y = find_peaks(Filter_Data * -1, height=0, distance=500)
    cardiac_cycle = np.diff(valley_x)
    peaks_x, peaks_y = find_peaks(Filter_Data, height=0, distance=500)
    peaks_y = peaks_y['peak_heights']
    valley_y = valley_y['peak_heights']
    return peaks_x, peaks_y, valley_x, valley_y, cardiac_cycle

In [6]:
def derivative(Data, Level, values=[]):
    result = np.gradient(Data)

    if Level == 0:
        values.append(Data)
        return 0
    else:
        values.append(Data)
        return derivative(result, Level - 1, values)
    
def process_wave(cycle):
    values = []
    derivative(cycle, 3, values)
    values = np.array(values)
    origin, derivative_1, derivative_2, derivative_3 = values
    derivative_1 = derivative_1 * 50
    derivative_2 = derivative_2 * 5000
    derivative_3 = derivative_3 * 100000

    return [derivative_1, derivative_2, derivative_3]

In [7]:
def Find_Path(path):

    File_path = []

    #find all Data_file path 
    for root,  subfolders,  filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('.txt'):
                continue
            filepath = root +'/'+ filename
            new_filepath = filepath.replace("\\", "/")
            File_path.append(new_filepath)

    return File_path

In [8]:
def get_Imformation(path,locate, imformation=[]):
    test=path.split('/')
    Name = test[locate[0]]
    Date = test[locate[1]]
    State_check = test[locate[2]]
    if State_check =='易堵':
        State = '0'
    else: 
        State = '1'

    file_name = test[len(test)-1]
    name_check = file_name.find('R')
    if name_check != -1:
        Status = 'Right'
    else:
        Status = 'Left'
    
    imformation =[Name, Date, State, Status]

    return imformation, Name

In [9]:
def plot_ppg(waveform, feature, Name, i, hands):
    if graph[0] != 'break':
        x = np.linspace(0, len(waveform), len(waveform))
        plt.figure(figsize=(5, 3))
        plt.plot(waveform, label='PPG')
        plt.plot(x[feature], waveform[feature], 'r.', label='peak')
        plt.title(f'{Name} PPG signal {i} {hands}', fontproperties=font_prop)
        plt.legend()
        plt.grid(True)
        if graph[1] == 'show':
            plt.show()
        elif graph[1] == 'save':
            plt.savefig(f'{graph_file}\\{Name}, {i + 1}th d1.jpg')

        plt.close('all')

def calculate_PPG(waveform, feature, Name, i, hands):
    B1,B10 = find_peaks(waveform, height=0.3, distance=500)
    
    B2 = waveform[feature[5]], waveform[feature[11]]
    valley,_ = find_peaks(-waveform, height=-0.3, distance=500)
    B3 = valley[1] - valley[0]
    y_section = np.abs(waveform[valley[0]:valley[1]+1])
    B5 = np.trapz(y_section, dx=1)
    B6 = B1[1] - B1[0]
    B7 = valley[-1] - B1[-1]
    B10 = B10['peak_heights']
    calculate_feature = [B1, B2, B3, B5, B6, B7, B10]
    print(f"B1(Systolic peak): {B1}, B1.5(Valley): {valley}, B2(Diastolic peak y): {B2}, B3(Cardiac Cycle): {B3}, B5(Cycle Area): {B5}, B6(SSI): {B6}, B7(Peak to Valley): {B7}, B10(Systolic peak y): {B10}")
    plot_ppg(waveform, B1, Name, i, hands)

def print_features(Name, i, hand, csv_path):
    # Read the CSV file
    data = pd.read_csv(csv_path, header=None)

    # Construct the key from Name, i, and hand to match the first column
    key = f"{Name}, {i}th {hand} d2"

    # Search for the matching row in the first column
    matching_row = data[data[0] == key]

    if not matching_row.empty:
        print(f"Features for {key}:")
        print(matching_row.iloc[0, 1:].values)  # Print feature values (ignoring the first column)
        return matching_row.iloc[0, 1:].values.tolist()
    else:
        print(f"No matching features found for {key}")
        return None

In [10]:
def plot_d1(waveform, Feature, Name, i, hands):
    if graph[1] != 'break':
        x = np.linspace(0, len(waveform), len(waveform))
        plt.figure()
        plt.plot(waveform)
        plt.plot(x[Feature], waveform[Feature], '*', label='Peak')
        plt.title(f'{Name}, {i + 1}th Left_Right',fontproperties=font_prop)
        plt.legend()
        plt.grid()
        if graph[1] == 'show':
            plt.show()
        elif graph[1] == 'save':
            plt.savefig(f'{graph_file}\\{Name}, {i + 1}th d1.jpg')

        plt.close('all')

def calculate_d1(waveform, Name, i, graph, hands):
    
    d1_peaks, B8 = find_peaks(waveform, height=0, distance=800)

    B4 = d1_peaks[1] - d1_peaks[0]
    B8 = B8['peak_heights']

    print(f"B4(1st cycle): {B4}, B8(1st peak): {B8}")


    if graph == 'show':
        plot_d1(waveform,d1_peaks , Name, i, hands )

In [11]:
def plot_d2(origin, derivative, Name, i, Feature, hands, locate):
    if graph[2] != 'break':
        x = np.linspace(0, len(derivative[1]), len(derivative[1]))
        plt.figure()
        
        plt.plot(origin)
        plt.plot(derivative[1])
        plt.plot(x[Feature], derivative[1][Feature], '*', label='Peak')
        plt.title(f'{Name}, {i + 1}th {hands}',fontproperties=font_prop)
        plt.legend()
        plt.grid()
        if graph[2] == 'show':
            plt.show()
        else:
            plt.savefig(f'{locate}\\{Name}, {i + 1}th {hands} d2.jpg')
        
        plt.close('all')

d2

In [12]:
def calculate_d2(origin, derivative, Name, i, hands):
    TDPPG_x = np.where(np.diff(np.sign(derivative[2])))[0]
    # 設置最小距離（例如，至少 10 個數據點）
    min_distance = 30

    def filter_by_distance(zero_crossings, min_distance):
        filtered_crossings = []
        last_index = -min_distance  # 初始化為一個遠距離的負數
        
        for index in zero_crossings:
            if index - last_index >= min_distance:
                filtered_crossings.append(index)
                last_index = index
        
        return np.array(filtered_crossings)

    # 過濾零點
    TDPPG_x = filter_by_distance(TDPPG_x, min_distance)

    closest_indices = []

    a_point = np.array(find_peaks(derivative[1], height=0.6, distance=900)[0])


    for num in a_point:
        differences = np.abs(TDPPG_x - num)
        closest_index = np.argmin(differences)
        closest_indices.append(closest_index)
    if len(closest_indices) < 2:  #判斷抓到兩個周期
        plot_d2(origin, derivative, Name, i, TDPPG_x, hands, bad_graph)
        print(f'{Name}, {i + 1}th {hands} cant find the feature')
        return 0, None
    else:
        TDPPG_x_new = TDPPG_x[closest_indices[0]:closest_indices[0]+6]
        TDPPG_x_new = np.append(TDPPG_x_new, TDPPG_x[closest_indices[1]:closest_indices[1]+6])
        if TDPPG_x_new[6] - TDPPG_x_new[5] < TDPPG_x_new[6] * 0.3 or len(TDPPG_x_new) != 12: #判斷是否抓錯feature
            plot_d2(origin, derivative, Name, i, TDPPG_x_new, hands, bad_graph)
            print(f'{Name}, {i + 1}th {hands} cant find the feature')
            return 0, None


    if graph =='show':
        plot_d2(origin,derivative, Name, i, TDPPG_x_new, hands, graph_file)
        print(TDPPG_x_new)
    else:
        file_path = f'{data_file}\\features.csv'

        # 創建一個包含 Name 和數據的 DataFrame
        new_data = pd.DataFrame([[f"{Name}, {i + 1}th {hands} d2"] + TDPPG_x_new.tolist()])
        try:
            with open(file_path, 'a', newline='') as f: # 如果檔案存在，追加模式 ('a') 並且不寫入頭
                new_data.to_csv(f, index=False, header=False)
        except FileNotFoundError: # 如果檔案不存在，創建新檔案並寫入頭
            new_data.to_csv(file_path, index=False, header=False)
        
        return 1, TDPPG_x_new
    
    return 0, None


In [13]:
def calculate_d2_Bad(derivative):
    derivative_3  = np.gradient(derivative) * 50
    d3_squ = np.square(derivative_3)
    
    threshold = 0.01  # 設定閾值
    square_wave = np.where(d3_squ < threshold, 1, 0)
    TDPPG_x = np.where(np.diff(square_wave))[0]
    print(TDPPG_x[0:10])

    # 給定的索引數據
    indexes = TDPPG_x[0:10]

    # 將索引兩兩分組
    grouped_indexes = indexes.reshape(-1, 2)
    # 保存結果
    result = []

    for start, end in grouped_indexes:
        if start == grouped_indexes[2][0]:
            result.append(start)
            result.append(end)
        else:
            midpoint = (start + end) // 2
            result.append(midpoint)

    # 輸出結果
    result = np.array(result)
    print("結果:", result)
    return result, square_wave, derivative_3

In [14]:
def resize_wave(waveform, Name, i, hands, target_length=2000): # type: ignore
    quilty = 1

    derivative = process_wave(waveform)

    resized = signal.resample(waveform, target_length)
    
    for j in range(3):
        derivative[j] = signal.resample(derivative[j], target_length)


    print(f"{Name}, {i}th {hands}")
    
    quilty, Feature = calculate_d2(resized, derivative, Name, i, hands) # type: ignore
    # if quilty == 1:
    #     calculate_d1(derivative[0], Feature, Name, i, hands)
    #     calculate_PPG(resized, Feature, Name, i, hands)
    if npy == 'save' and quilty == 1:
        np.save(f'{data_file}\\{Name}, {i + 1}th {hands}.npy', resized)
        np.save(f'{data_file}\\{Name}, {i + 1}th {hands} d1.npy', derivative[0])
        np.save(f'{data_file}\\{Name}, {i + 1}th {hands} d2.npy', derivative[1])

    elif npy == 'save' and quilty == 0:
        np.save(f'{data_file}\\bad signal\\{Name}, {i + 1}th {hands}.npy', resized)
        np.save(f'{data_file}\\bad signal\\{Name}, {i + 1}th {hands} d1.npy', derivative[0])
        np.save(f'{data_file}\\bad signal\\{Name}, {i + 1}th {hands} d2.npy', derivative[1])

In [15]:
def main(): #!for folder data
    channel1_id = 2
    channel2_id = 4
    record_id = 1

    if data == 'Normal':
        File_path = Find_Path("F:\\正常人Data") #!正常人
    elif data == 'Patient':
        File_path = Find_Path("F:\\第二次收案") #!病患
    print('找到資料筆數', len(File_path))

    df_c = pd.DataFrame()
    for j, path in tqdm(enumerate(File_path), total=len(File_path), desc='Processing'):
        Data = adi.read_file(path)

        Right = Data.channels[channel1_id - 1].get_data(record_id)
        Left = Data.channels[channel2_id - 1].get_data(record_id)

        Filter_Left,Filter_Right = butter(Left, Right, 0.5, 9, 1000, 4)

        L_wave = Filter_Left[20000:300000] * 10
        R_wave = Filter_Right[20000:300000] * 10

        L_valley_x, L_valley_y = find_peaks(L_wave * -1, height=0, distance=150)
        R_valley_x, R_valley_y = find_peaks(R_wave * -1, height=0, distance=150)

        L_valley_y = L_valley_y['peak_heights']
        R_valley_y = R_valley_y['peak_heights']

        if data == 'Normal':
            Imformation,Name = get_Imformation(path,locate=[2,3,4]) #!正常人
        elif data == 'Patient':
            Imformation,Name = get_Imformation(path,locate=[3,4,2]) #!病患

        if len(L_valley_x) > len(R_valley_x): #找最小的cycle
            min_cycle = len(R_valley_x)
        else:
            min_cycle = len(L_valley_x)
        
        for i in range(0,min_cycle - 2,2):
            diff = np.abs(L_valley_x[i] - R_valley_x[i]) #time diff
            L_cycle = L_wave[L_valley_x[i]:L_valley_x[i + 2]] #two cycle
            L_cycle_cut = [L_wave[L_valley_x[i]:L_valley_x[i + 1]], L_wave[L_valley_x[i + 1]:L_valley_x[i + 2]]] #divide 2

            L_peaks_x, L_peaks_y = find_peaks(L_cycle, height=0, distance=500)
            L_peaks_y = L_peaks_y['peak_heights']
            L_peak = [L_peaks_x, L_peaks_y]

            R_cycle = R_wave[L_valley_x[i]:L_valley_x[i + 2]]
            R_cycle_cut = [R_wave[R_valley_x[i]:R_valley_x[i + 1]], R_wave[R_valley_x[i + 1]:R_valley_x[i + 2]]]  # vivide 2

            R_peaks_x, R_peaks_y = find_peaks(R_cycle, height=0, distance=500)
            R_peaks_y = R_peaks_y['peak_heights']
            R_peak = [R_peaks_x, R_peaks_y]

            if len(L_cycle) < 1100 or len(L_peaks_y) != 2 or len(R_peaks_y) != 2 or len(R_cycle) < 1100:
                continue

            if L_peaks_y[0] < 0.5:
                L_cycle *= 0.5 / L_peaks_y[0]
                L_peaks_y[0] = 0.5
                L_peaks_y[1] = 0.5

            if R_peaks_y[0] < 0.5:
                R_cycle *= 0.5 / R_peaks_y[0]
                R_peaks_y[0] = 0.5
                R_peaks_y[1] = 0.5
            resize_wave(L_cycle, Name, i,"Left", 2000 )
            resize_wave(R_cycle, Name, i, "Right", 2000 )



In [16]:
# def main(): #!for single data
#     channel1_id = 2
#     channel2_id = 4
#     record_id = 1

#     Name = 'test'
    
#     path = "F:\\正常人Data\\Normal1\\四下午\\Normal1.adicht"
#     Data = adi.read_file(path)

#     Right = Data.channels[channel1_id - 1].get_data(record_id)
#     Left = Data.channels[channel2_id - 1].get_data(record_id)

#     Filter_Left,Filter_Right = butter(Left, Right, 0.5, 9, 1000, 4)

#     L_wave = Filter_Left[100000:120000] * 10
#     R_wave = Filter_Right[100000:120000] * 10

#     L_valley_x, L_valley_y = find_peaks(L_wave * -1, height=0, distance=150)
#     R_valley_x, R_valley_y = find_peaks(R_wave * -1, height=0, distance=150)

#     L_valley_y = L_valley_y['peak_heights']
#     R_valley_y = R_valley_y['peak_heights']

#     if len(L_valley_x) > len(R_valley_x): #找最小的cycle
#         min_cycle = len(R_valley_x)
#     else:
#         min_cycle = len(L_valley_x)
    
#     for i in range(0,min_cycle - 2,2):
#         diff = np.abs(L_valley_x[i] - R_valley_x[i]) #time diff
#         L_cycle = L_wave[L_valley_x[i]:L_valley_x[i + 2]] #two cycle
#         L_cycle_cut = [L_wave[L_valley_x[i]:L_valley_x[i + 1]], L_wave[L_valley_x[i + 1]:L_valley_x[i + 2]]] #divide 2

#         L_peaks_x, L_peaks_y = find_peaks(L_cycle, height=0, distance=500)
#         L_peaks_y = L_peaks_y['peak_heights']
#         L_peak = [L_peaks_x, L_peaks_y]

#         R_cycle = R_wave[L_valley_x[i]:L_valley_x[i + 2]]
#         R_cycle_cut = [R_wave[R_valley_x[i]:R_valley_x[i + 1]], R_wave[R_valley_x[i + 1]:R_valley_x[i + 2]]]  # vivide 2

#         R_peaks_x, R_peaks_y = find_peaks(R_cycle, height=0, distance=500)
#         R_peaks_y = R_peaks_y['peak_heights']
#         R_peak = [R_peaks_x, R_peaks_y]

#         if len(L_cycle) < 1100 or len(L_peaks_y) != 2 or len(R_peaks_y) != 2 or len(R_cycle) < 1100:
#             continue

#         if L_peaks_y[0] < 0.5:
#             L_cycle *= 0.5 / L_peaks_y[0]
#             L_peaks_y[0] = 0.5
#             L_peaks_y[1] = 0.5

#         if R_peaks_y[0] < 0.5:
#             R_cycle *= 0.5 / R_peaks_y[0]
#             R_peaks_y[0] = 0.5
#             R_peaks_y[1] = 0.5
        

#         resize_wave(L_cycle, R_cycle, Name, i, 2000)


: 

In [ ]:
if __name__ == '__main__':
    main()

找到資料筆數 63


Processing:   0%|          | 0/63 [00:00<?, ?it/s]

Normal1, 0th Left
Normal1, 0th Right
Normal1, 2th Left
Normal1, 2th Right
Normal1, 4th Left
Normal1, 4th Right
Normal1, 6th Left
Normal1, 6th Right
Normal1, 8th Left
Normal1, 8th Right
Normal1, 10th Left
Normal1, 10th Right
Normal1, 12th Left
Normal1, 12th Right
Normal1, 14th Left
Normal1, 14th Right
Normal1, 16th Left
Normal1, 16th Right
Normal1, 18th Left
Normal1, 18th Right
Normal1, 20th Left
Normal1, 20th Right
Normal1, 22th Left
Normal1, 22th Right
Normal1, 24th Left
Normal1, 24th Right
Normal1, 26th Left
Normal1, 26th Right
Normal1, 28th Left
Normal1, 28th Right
Normal1, 30th Left
Normal1, 30th Right
Normal1, 32th Left
Normal1, 32th Right
Normal1, 34th Left
Normal1, 34th Right
Normal1, 36th Left
Normal1, 36th Right
Normal1, 38th Left
Normal1, 38th Right
Normal1, 40th Left
Normal1, 40th Right
Normal1, 42th Left
Normal1, 42th Right
Normal1, 44th Left
Normal1, 44th Right
Normal1, 46th Left
Normal1, 46th Right
Normal1, 48th Left
Normal1, 48th Right
Normal1, 50th Left
Normal1, 50th Ri

Processing:   2%|▏         | 1/63 [00:00<00:46,  1.34it/s]

Normal1, 262th Right
Normal1, 264th Left
Normal1, 264th Right
Normal1, 266th Left
Normal1, 266th Right
Normal1, 268th Left
Normal1, 268th Right
Normal1, 269th Right cant find the feature
Normal1, 270th Left
Normal1, 270th Right
Normal1, 272th Left
Normal1, 272th Right
Normal1, 273th Right cant find the feature
Normal1, 274th Left
Normal1, 274th Right
Normal1, 276th Left
Normal1, 276th Right
Normal1, 278th Left
Normal1, 278th Right
Normal1, 280th Left
Normal1, 280th Right
Normal1, 281th Right cant find the feature
Normal1, 282th Left
Normal1, 282th Right
Normal1, 284th Left
Normal1, 285th Left cant find the feature
Normal1, 284th Right
Normal1, 285th Right cant find the feature
Normal1, 286th Left
Normal1, 287th Left cant find the feature
Normal1, 286th Right
Normal1, 287th Right cant find the feature
Normal1, 288th Left
Normal1, 288th Right
Normal1, 289th Right cant find the feature
Normal1, 290th Left
Normal1, 290th Right
Normal1, 292th Left
Normal1, 292th Right
Normal1, 294th Left
No

...

Normal50, 30th Right
Normal50, 44th Left
Normal50, 45th Left cant find the feature
Normal50, 44th Right
Normal50, 112th Left
Normal50, 113th Left cant find the feature
Normal50, 112th Right
Normal50, 164th Left
Normal50, 165th Left cant find the feature
Normal50, 164th Right
Normal50, 494th Left
Normal50, 495th Left cant find the feature
Normal50, 494th Right
Normal50, 650th Left
Normal50, 651th Left cant find the feature
Normal50, 650th Right
Normal50, 668th Left
Normal50, 669th Left cant find the feature
Normal50, 668th Right
Normal50, 674th Left
Normal50, 675th Left cant find the feature
Normal50, 674th Right
Normal50, 676th Left
Normal50, 676th Right
Normal50, 684th Left
Normal50, 685th Left cant find the feature
Normal50, 684th Right
Normal50, 704th Left
Normal50, 705th Left cant find the feature
Normal50, 704th Right
Normal50, 710th Left
Normal50, 711th Left cant find the feature
Normal50, 710th Right
Normal50, 718th Left
Normal50, 719th Left cant find the feature
Normal50, 718th

Processing:  71%|███████▏  | 45/63 [00:34<00:12,  1.48it/s]

Normal51, 370th Left
Normal51, 370th Right
Normal51, 372th Left
Normal51, 372th Right
Normal51, 373th Right cant find the feature
Normal51, 374th Left
Normal51, 374th Right
Normal51, 376th Left
Normal51, 376th Right
Normal51, 378th Left
Normal51, 378th Right
Normal51, 380th Left
Normal51, 381th Left cant find the feature
Normal51, 380th Right
Normal51, 381th Right cant find the feature
Normal51, 382th Left
Normal51, 383th Left cant find the feature
Normal51, 382th Right
Normal51, 383th Right cant find the feature
Normal51, 384th Left
Normal51, 385th Left cant find the feature
Normal51, 384th Right
Normal51, 386th Left
Normal51, 386th Right
Normal51, 388th Left
Normal51, 388th Right
Normal51, 390th Left
Normal51, 390th Right
Normal51, 392th Left
Normal51, 392th Right
Normal51, 394th Left
Normal51, 394th Right
Normal51, 396th Left
Normal51, 397th Left cant find the feature
Normal51, 396th Right
Normal51, 397th Right cant find the feature
Normal51, 398th Left
Normal51, 398th Right
Normal5

Processing:  73%|███████▎  | 46/63 [00:35<00:12,  1.35it/s]

Normal52, 302th Right
Normal52, 304th Left
Normal52, 304th Right
Normal52, 306th Left
Normal52, 306th Right
Normal53, 0th Left
Normal53, 0th Right
Normal53, 2th Left
Normal53, 2th Right
Normal53, 4th Left
Normal53, 4th Right
Normal53, 6th Left
Normal53, 6th Right
Normal53, 8th Left
Normal53, 8th Right
Normal53, 10th Left
Normal53, 10th Right
Normal53, 12th Left
Normal53, 12th Right
Normal53, 14th Left
Normal53, 14th Right
Normal53, 16th Left
Normal53, 16th Right
Normal53, 18th Left
Normal53, 18th Right
Normal53, 20th Left
Normal53, 20th Right
Normal53, 22th Left
Normal53, 22th Right
Normal53, 24th Left
Normal53, 24th Right
Normal53, 26th Left
Normal53, 26th Right
Normal53, 28th Left
Normal53, 28th Right
Normal53, 30th Left
Normal53, 30th Right
Normal53, 32th Left
Normal53, 32th Right
Normal53, 34th Left
Normal53, 34th Right
Normal53, 36th Left
Normal53, 36th Right
Normal53, 38th Left
Normal53, 38th Right
Normal53, 40th Left
Normal53, 40th Right
Normal53, 42th Left
Normal53, 42th Right


In [ ]:
'''
65 3th
16 7th
16 1th
15 1th
32 7th
31 3th
29 5th
55 all
51 all
5 all
48 all
42 can't use 
41 can't use
3 all
27 all

'''